# Admin Dashboard

This notebook creates an interactive admin dashboard for hotel booking analytics. It provides insights on user behavior and sales metrics in an easy-to-use interface.

## Import Required Libraries

In [ ]:
# Import libraries for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import datetime as dt

# Set visualization styles
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")

## Load and Prepare Data

In [ ]:
# Load the hotel booking dataset
# Assuming the dataset is in the same directory
try:
    df = pd.read_csv('hotel_bookings.csv')
    print("Dataset loaded successfully!")
    print(f"Shape of the dataset: {df.shape}")
    print("\nFirst 5 rows of the dataset:")
    print(df.head())
except FileNotFoundError:
    print("Dataset file not found. Please check the file path.")
    
    # Create sample data for demonstration purposes if file not found
    print("\nCreating sample data for demonstration...")
    
    # Sample data generation
    np.random.seed(42)
    dates = pd.date_range(start='2022-01-01', end='2023-01-01', freq='D')
    
    sample_data = {
        'booking_date': np.random.choice(dates, 1000),
        'user_id': np.random.randint(1, 201, 1000),
        'hotel_type': np.random.choice(['Resort Hotel', 'City Hotel'], 1000),
        'is_canceled': np.random.choice([0, 1], 1000, p=[0.7, 0.3]),
        'lead_time': np.random.randint(0, 365, 1000),
        'arrival_date': np.random.choice(dates, 1000),
        'nights': np.random.randint(1, 15, 1000),
        'adults': np.random.randint(1, 4, 1000),
        'children': np.random.choice([0, 1, 2], 1000, p=[0.6, 0.3, 0.1]),
        'meal_type': np.random.choice(['BB', 'HB', 'FB'], 1000),
        'country': np.random.choice(['USA', 'GBR', 'FRA', 'ESP', 'ITA', 'CHN'], 1000),
        'adr': np.random.uniform(50, 300, 1000)  # Average Daily Rate (price)
    }
    
    df = pd.DataFrame(sample_data)
    
    # Calculate total price
    df['total_price'] = df['adr'] * df['nights'] * (df['adults'] + 0.5 * df['children'])
    
    print("\nSample data created!")
    print(f"Shape of the sample dataset: {df.shape}")
    print("\nFirst 5 rows of the sample dataset:")
    print(df.head())

In [ ]:
# Data cleaning and preparation
# Handle missing values
print("Missing values in the dataset:")
print(df.isnull().sum())

# Fill missing values or drop rows with missing values if necessary
df = df.dropna(subset=['user_id', 'booking_date', 'hotel_type'])  # Drop rows with missing critical data
df = df.fillna({
    'children': 0,  
    'country': 'Unknown'
})

# Convert date columns to datetime
df['booking_date'] = pd.to_datetime(df['booking_date'])
df['arrival_date'] = pd.to_datetime(df['arrival_date'])

# Extract month and year for time-based analysis
df['booking_month'] = df['booking_date'].dt.month
df['booking_year'] = df['booking_date'].dt.year
df['arrival_month'] = df['arrival_date'].dt.month
df['arrival_year'] = df['arrival_date'].dt.year

# Create additional metrics
df['advance_booking_days'] = (df['arrival_date'] - df['booking_date']).dt.days
df['is_weekend_arrival'] = df['arrival_date'].dt.dayofweek >= 5  # 5=Sat, 6=Sun

print("\nDataset after cleaning:")
print(df.info())
print("\nSample of processed data:")
print(df.head())

## Create User Analytics Section

In [ ]:
# Analyze user data
# Count unique users
unique_users = df['user_id'].nunique()
print(f"Total number of unique users: {unique_users}")

# Monthly active users
monthly_users = df.groupby(['booking_year', 'booking_month'])['user_id'].nunique().reset_index()
monthly_users.columns = ['Year', 'Month', 'Active Users']

# Create date column for better visualization
monthly_users['Date'] = pd.to_datetime(monthly_users['Year'].astype(str) + '-' + 
                                      monthly_users['Month'].astype(str) + '-01')

# Plot monthly active users
fig_mau = px.line(monthly_users, x='Date', y='Active Users', 
                 title='Monthly Active Users',
                 labels={'Date': 'Month', 'Active Users': 'Number of Active Users'})
fig_mau.update_xaxes(tickformat="%b %Y")
fig_mau.show()

# User retention analysis (simplified)
# Get first booking date for each user
first_bookings = df.groupby('user_id')['booking_date'].min().reset_index()
first_bookings.columns = ['user_id', 'first_booking_date']

# Merge with original data
df_retention = pd.merge(df, first_bookings, on='user_id')

# Calculate days since first booking
df_retention['days_since_first_booking'] = (df_retention['booking_date'] - df_retention['first_booking_date']).dt.days

# Group users into cohorts (monthly)
df_retention['cohort'] = df_retention['first_booking_date'].dt.to_period('M')

# Count unique users by cohort and booking month
cohort_data = df_retention.groupby(['cohort', df_retention['booking_date'].dt.to_period('M')])['user_id'].nunique().reset_index()
cohort_data.columns = ['Cohort', 'Booking Month', 'Users']

# Calculate retention periods (months since first booking)
cohort_data['Period'] = (cohort_data['Booking Month'] - cohort_data['Cohort']).apply(lambda x: x.n)

# Filter to show only the first 6 cohorts and periods
cohort_pivot = cohort_data[cohort_data['Period'] >= 0].pivot_table(index='Cohort', columns='Period', values='Users')
retention_rates = cohort_pivot.divide(cohort_pivot[0], axis=0) * 100

# Display retention rates
print("User Retention Rates (%):")
print(retention_rates.round(2).head(6))

# Plot retention heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(retention_rates.head(6), annot=True, fmt='.1f', cmap='YlGnBu')
plt.title('User Retention by Cohort (%)')
plt.xlabel('Months Since First Booking')
plt.ylabel('Cohort')
plt.show()

## Create Sales Analytics Section

In [ ]:
# Sales analytics
# Calculate overall booking metrics
total_bookings = len(df)
canceled_bookings = df['is_canceled'].sum()
canceled_rate = (canceled_bookings / total_bookings) * 100
completed_bookings = total_bookings - canceled_bookings

# Calculate revenue metrics (using ADR or calculated total_price)
if 'total_price' in df.columns:
    total_revenue = df[df['is_canceled'] == 0]['total_price'].sum()
    avg_booking_value = df[df['is_canceled'] == 0]['total_price'].mean()
else:
    # Calculate estimated revenue based on ADR, nights and number of guests
    df['total_price'] = df['adr'] * df['nights'] * (df['adults'] + 0.5 * df['children'])
    total_revenue = df[df['is_canceled'] == 0]['total_price'].sum()
    avg_booking_value = df[df['is_canceled'] == 0]['total_price'].mean()

# Print key metrics
print(f"Total Bookings: {total_bookings}")
print(f"Completed Bookings: {completed_bookings} ({100 - canceled_rate:.1f}%)")
print(f"Canceled Bookings: {canceled_bookings} ({canceled_rate:.1f}%)")
print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Average Booking Value: ${avg_booking_value:.2f}")

# Monthly booking and revenue trends
monthly_bookings = df.groupby(['booking_year', 'booking_month']).size().reset_index(name='Bookings')
monthly_bookings['Date'] = pd.to_datetime(monthly_bookings['booking_year'].astype(str) + '-' + 
                                         monthly_bookings['booking_month'].astype(str) + '-01')

monthly_revenue = df[df['is_canceled'] == 0].groupby(['booking_year', 'booking_month'])['total_price'].sum().reset_index()
monthly_revenue['Date'] = pd.to_datetime(monthly_revenue['booking_year'].astype(str) + '-' + 
                                        monthly_revenue['booking_month'].astype(str) + '-01')

# Create a subplot with two y-axes
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=monthly_bookings['Date'], y=monthly_bookings['Bookings'], name="Bookings"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=monthly_revenue['Date'], y=monthly_revenue['total_price'], name="Revenue"),
    secondary_y=True,
)

# Set x-axis title
fig.update_xaxes(title_text="Date", tickformat="%b %Y")

# Set y-axes titles
fig.update_yaxes(title_text="Number of Bookings", secondary_y=False)
fig.update_yaxes(title_text="Revenue ($)", secondary_y=True)

fig.update_layout(title_text="Monthly Bookings and Revenue")
fig.show()

# Hotel type performance comparison
hotel_performance = df.groupby('hotel_type').agg({
    'booking_date': 'count',
    'is_canceled': 'mean',
    'total_price': lambda x: x[df['is_canceled'] == 0].sum(),
    'adr': 'mean'
}).reset_index()

hotel_performance.columns = ['Hotel Type', 'Total Bookings', 'Cancellation Rate', 'Total Revenue', 'Average Daily Rate']
hotel_performance['Cancellation Rate'] = hotel_performance['Cancellation Rate'] * 100

print("\nHotel Type Performance:")
print(hotel_performance)

# Plot hotel type comparison
fig = px.bar(hotel_performance, x='Hotel Type', y='Total Revenue', 
             color='Hotel Type', 
             hover_data=['Total Bookings', 'Cancellation Rate', 'Average Daily Rate'],
             title='Revenue by Hotel Type')
fig.show()

## Visualize Key Metrics

In [ ]:
# Create visualizations for key metrics

# 1. Booking distribution by country (top 10)
country_bookings = df['country'].value_counts().nlargest(10).reset_index()
country_bookings.columns = ['Country', 'Number of Bookings']

fig_countries = px.bar(country_bookings, 
                       x='Country', 
                       y='Number of Bookings',
                       title='Top 10 Countries by Number of Bookings',
                       color='Number of Bookings')
fig_countries.show()

# 2. Booking distribution by month
monthly_dist = df.groupby(['booking_month']).size().reset_index(name='Count')
monthly_dist['Month'] = pd.Categorical(monthly_dist['booking_month'], 
                                      categories=range(1, 13), 
                                      ordered=True)
monthly_dist['Month Name'] = monthly_dist['booking_month'].apply(lambda x: dt.date(2000, x, 1).strftime('%B'))

fig_monthly = px.line(monthly_dist, 
                     x='Month Name', 
                     y='Count',
                     title='Booking Distribution by Month',
                     markers=True)
fig_monthly.update_xaxes(categoryorder='array', 
                        categoryarray=[dt.date(2000, i, 1).strftime('%B') for i in range(1, 13)])
fig_monthly.show()

# 3. Lead time analysis (advance booking days)
fig_lead = px.histogram(df, 
                       x='advance_booking_days',
                       nbins=30,
                       title='Distribution of Advance Booking Days',
                       color_discrete_sequence=['skyblue'])
fig_lead.update_layout(xaxis_title='Days in Advance', yaxis_title='Number of Bookings')
fig_lead.show()

# 4. Cancellation rate by lead time
lead_time_bins = [0, 7, 30, 90, 180, 365, df['advance_booking_days'].max()]
df['lead_time_category'] = pd.cut(df['advance_booking_days'], bins=lead_time_bins, 
                                 labels=['0-7 days', '8-30 days', '31-90 days', 
                                         '91-180 days', '181-365 days', '365+ days'])

cancel_by_lead = df.groupby('lead_time_category').agg({
    'is_canceled': ['mean', 'count']
}).reset_index()
cancel_by_lead.columns = ['Lead Time', 'Cancellation Rate', 'Count']
cancel_by_lead['Cancellation Rate'] = cancel_by_lead['Cancellation Rate'] * 100

fig_cancel_lead = px.bar(cancel_by_lead, 
                        x='Lead Time', 
                        y='Cancellation Rate',
                        title='Cancellation Rate by Lead Time',
                        text_auto='.1f',
                        color='Cancellation Rate')
fig_cancel_lead.update_traces(texttemplate='%{text}%', textposition='outside')
fig_cancel_lead.update_layout(yaxis_title='Cancellation Rate (%)')
fig_cancel_lead.show()

# 5. Average daily rate by month
adr_by_month = df.groupby(['booking_month'])['adr'].mean().reset_index()
adr_by_month['Month Name'] = adr_by_month['booking_month'].apply(lambda x: dt.date(2000, x, 1).strftime('%B'))

fig_adr = px.line(adr_by_month, 
                 x='Month Name', 
                 y='adr',
                 title='Average Daily Rate by Month',
                 markers=True)
fig_adr.update_xaxes(categoryorder='array', 
                    categoryarray=[dt.date(2000, i, 1).strftime('%B') for i in range(1, 13)])
fig_adr.update_layout(yaxis_title='Average Daily Rate ($)', xaxis_title='Month')
fig_adr.show()

# 6. Guest composition (adults vs children)
df['total_guests'] = df['adults'] + df['children']
guest_composition = df.groupby('total_guests').agg({
    'booking_date': 'count',
    'adults': 'mean',
    'children': 'mean'
}).reset_index()
guest_composition.columns = ['Total Guests', 'Count', 'Average Adults', 'Average Children']
guest_composition = guest_composition[guest_composition['Total Guests'] <= 5]  # Filter outliers

fig_composition = px.bar(guest_composition, 
                        x='Total Guests', 
                        y=['Average Adults', 'Average Children'],
                        title='Guest Composition by Party Size',
                        barmode='stack')
fig_composition.update_layout(xaxis_title='Total Number of Guests', yaxis_title='Average Number of Guests')
fig_composition.show()

## Build Interactive Dashboard

In [ ]:
# Create an interactive dashboard using Dash

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1('Hotel Booking Admin Dashboard', style={'textAlign': 'center', 'color': '#2c3e50'}),
    
    html.Div([
        html.H2('Filters', style={'color': '#34495e'}),
        html.Div([
            html.Label('Select Date Range:'),
            dcc.DatePickerRange(
                id='date-range',
                start_date=df['booking_date'].min().date(),
                end_date=df['booking_date'].max().date(),
                display_format='MMM DD, YYYY'
            )
        ], style={'marginBottom': 20}),
        
        html.Div([
            html.Label('Select Hotel Type:'),
            dcc.Dropdown(
                id='hotel-type',
                options=[{'label': hotel, 'value': hotel} for hotel in df['hotel_type'].unique()],
                value=df['hotel_type'].unique().tolist(),  # Default select all
                multi=True
            )
        ], style={'marginBottom': 20})
    ], style={'padding': '20px', 'backgroundColor': '#f8f9fa', 'borderRadius': '10px', 'margin': '10px'}),
    
    # Key metrics section
    html.Div([
        html.H2('Key Metrics', style={'color': '#34495e', 'textAlign': 'center'}),
        html.Div([
            html.Div([
                html.H3('Total Bookings', style={'textAlign': 'center'}),
                html.P(id='total-bookings', style={'textAlign': 'center', 'fontSize': 24, 'fontWeight': 'bold'})
            ], className='metric-card', style={'width': '24%', 'display': 'inline-block', 'backgroundColor': '#ecf0f1', 'padding': '10px', 'borderRadius': '5px'}),
            
            html.Div([
                html.H3('Completed Bookings', style={'textAlign': 'center'}),
                html.P(id='completed-bookings', style={'textAlign': 'center', 'fontSize': 24, 'fontWeight': 'bold'})
            ], className='metric-card', style={'width': '24%', 'display': 'inline-block', 'backgroundColor': '#ecf0f1', 'padding': '10px', 'borderRadius': '5px', 'marginLeft': '1%'}),
            
            html.Div([
                html.H3('Total Revenue', style={'textAlign': 'center'}),
                html.P(id='total-revenue', style={'textAlign': 'center', 'fontSize': 24, 'fontWeight': 'bold'})
            ], className='metric-card', style={'width': '24%', 'display': 'inline-block', 'backgroundColor': '#ecf0f1', 'padding': '10px', 'borderRadius': '5px', 'marginLeft': '1%'}),
            
            html.Div([
                html.H3('Avg Booking Value', style={'textAlign': 'center'}),
                html.P(id='avg-booking', style={'textAlign': 'center', 'fontSize': 24, 'fontWeight': 'bold'})
            ], className='metric-card', style={'width': '24%', 'display': 'inline-block', 'backgroundColor': '#ecf0f1', 'padding': '10px', 'borderRadius': '5px', 'marginLeft': '1%'})
        ])
    ], style={'padding': '20px', 'backgroundColor': '#ffffff', 'borderRadius': '10px', 'margin': '10px'}),
    
    # Charts section
    html.Div([
        html.Div([
            html.H2('Booking Trends', style={'color': '#34495e', 'textAlign': 'center'}),
            dcc.Graph(id='booking-trend')
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}),
        
        html.Div([
            html.H2('Revenue Analysis', style={'color': '#34495e', 'textAlign': 'center'}),
            dcc.Graph(id='revenue-analysis')
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'})
    ], style={'padding': '20px', 'backgroundColor': '#ffffff', 'borderRadius': '10px', 'margin': '10px'}),
    
    # More charts
    html.Div([
        html.Div([
            html.H2('Cancellation Analysis', style={'color': '#34495e', 'textAlign': 'center'}),
            dcc.Graph(id='cancel-chart')
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}),
        
        html.Div([
            html.H2('Country Distribution', style={'color': '#34495e', 'textAlign': 'center'}),
            dcc.Graph(id='country-chart')
        ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px'})
    ], style={'padding': '20px', 'backgroundColor': '#ffffff', 'borderRadius': '10px', 'margin': '10px'})
])

# Define callbacks to update dashboard components based on filters
@app.callback(
    [Output('total-bookings', 'children'),
     Output('completed-bookings', 'children'),
     Output('total-revenue', 'children'),
     Output('avg-booking', 'children'),
     Output('booking-trend', 'figure'),
     Output('revenue-analysis', 'figure'),
     Output('cancel-chart', 'figure'),
     Output('country-chart', 'figure')],
    [Input('date-range', 'start_date'),
     Input('date-range', 'end_date'),
     Input('hotel-type', 'value')]
)
def update_dashboard(start_date, end_date, hotel_types):
    # Filter data based on inputs
    filtered_df = df.copy()
    
    # Convert string dates to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Apply filters
    filtered_df = filtered_df[(filtered_df['booking_date'] >= start_date) & 
                              (filtered_df['booking_date'] <= end_date)]
    
    if hotel_types:
        filtered_df = filtered_df[filtered_df['hotel_type'].isin(hotel_types)]
    
    # Calculate metrics
    total_bookings = len(filtered_df)
    completed_bookings = len(filtered_df[filtered_df['is_canceled'] == 0])
    total_revenue = filtered_df[filtered_df['is_canceled'] == 0]['total_price'].sum()
    avg_booking_value = filtered_df[filtered_df['is_canceled'] == 0]['total_price'].mean()
    
    # Create booking trend chart
    monthly_data = filtered_df.groupby(pd.Grouper(key='booking_date', freq='M')).size().reset_index(name='Bookings')
    booking_trend_fig = px.line(monthly_data, x='booking_date', y='Bookings',
                             title='Monthly Booking Trend',
                             labels={'booking_date': 'Date', 'Bookings': 'Number of Bookings'})
    booking_trend_fig.update_xaxes(tickformat="%b %Y")
    
    # Create revenue analysis chart
    revenue_by_type = filtered_df[filtered_df['is_canceled'] == 0].groupby('hotel_type')['total_price'].sum().reset_index()
    revenue_fig = px.pie(revenue_by_type, values='total_price', names='hotel_type', 
                       title='Revenue Distribution by Hotel Type',
                       hole=0.3)
    
    # Create cancellation analysis
    cancel_df = filtered_df.groupby('hotel_type').agg({
        'is_canceled': 'mean',
        'booking_date': 'count'
    }).reset_index()
    cancel_df.columns = ['Hotel Type', 'Cancellation Rate', 'Total Bookings']
    cancel_df['Cancellation Rate'] = cancel_df['Cancellation Rate'] * 100
    
    cancel_fig = px.bar(cancel_df, x='Hotel Type', y='Cancellation Rate',
                       title='Cancellation Rate by Hotel Type',
                       color='Hotel Type',
                       text_auto='.1f')
    cancel_fig.update_traces(texttemplate='%{text}%', textposition='outside')
    
    # Create country distribution chart
    country_data = filtered_df['country'].value_counts().nlargest(10).reset_index()
    country_data.columns = ['Country', 'Count']
    
    country_fig = px.bar(country_data, x='Country', y='Count',
                        title='Top 10 Countries',
                        color='Count')
    
    return (
        f"{total_bookings:,}",
        f"{completed_bookings:,} ({completed_bookings/total_bookings*100:.1f}%)",
        f"${total_revenue:,.2f}",
        f"${avg_booking_value:.2f}",
        booking_trend_fig,
        revenue_fig,
        cancel_fig,
        country_fig
    )

# Run the server
if __name__ == '__main__':
    print("Dashboard is ready! Run this cell to start the interactive dashboard.")
    # Uncommenting the line below would start the dashboard in a new browser tab
    # app.run_server(debug=True)
    
    # For Jupyter notebook, you can use JupyterDash for better integration
    print("Note: In a production environment, replace with app.run_server(debug=False)")

## Conclusion

This admin dashboard provides a comprehensive view of the hotel booking data with interactive filters and visualizations. Key features include:

1. **User Analytics**: Active user tracking and cohort analysis for user retention patterns
2. **Sales Analytics**: Revenue trends, booking patterns, and cancellation rates
3. **Visual Metrics**: Interactive charts showing booking distribution by country, monthly trends, etc.
4. **Interactive Dashboard**: A filterable dashboard for exploring data by date ranges and hotel types

To extend this dashboard:
- Connect it to a live database for real-time updates
- Add user authentication for admin access
- Implement predictive analytics for forecasting future bookings and revenue
- Add email reporting functionality to schedule regular reports

# Admin Dashboard

This notebook creates an interactive admin dashboard for hotel booking data analysis. The dashboard will include user statistics, key metrics visualizations, and interactive filters.

## Import Required Libraries

In [ ]:
# Import libraries for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import datetime as dt

# Set styles for better visualization
plt.style.use('seaborn')
sns.set_palette('viridis')

## Load and Prepare Data

In [ ]:
# Load the dataset (assuming we have a hotel bookings dataset)
# Replace this path with the actual path to your data
try:
    df = pd.read_csv('../data/hotel_bookings.csv')
    print("Data loaded successfully!")
except FileNotFoundError:
    print("Data file not found. Using sample data instead.")
    # Create sample data for demonstration
    np.random.seed(42)
    dates = pd.date_range(start='2020-01-01', end='2022-12-31', freq='D')
    
    # Sample data generation
    df = pd.DataFrame({
        'booking_date': np.random.choice(dates, 5000),
        'hotel_type': np.random.choice(['City Hotel', 'Resort Hotel'], 5000),
        'lead_time': np.random.randint(0, 365, 5000),
        'arrival_date': np.random.choice(dates, 5000),
        'stays_in_nights': np.random.randint(1, 15, 5000),
        'adults': np.random.randint(1, 4, 5000),
        'children': np.random.randint(0, 3, 5000),
        'booking_status': np.random.choice(['Confirmed', 'Canceled', 'Check-Out'], 5000, p=[0.6, 0.3, 0.1]),
        'room_type': np.random.choice(['Standard', 'Deluxe', 'Suite', 'Premium'], 5000),
        'daily_rate': np.random.uniform(50, 500, 5000).round(2),
        'customer_type': np.random.choice(['Transient', 'Contract', 'Group', 'Transient-Party'], 5000),
        'country': np.random.choice(['USA', 'GBR', 'FRA', 'ESP', 'DEU', 'ITA', 'CHN', 'JPN'], 5000),
        'market_segment': np.random.choice(['Online TA', 'Offline TA', 'Direct', 'Corporate', 'Groups'], 5000),
        'deposit_type': np.random.choice(['No Deposit', 'Non Refund', 'Refundable'], 5000),
        'customer_id': np.random.randint(10000, 99999, 5000)
    })

In [ ]:
# Clean and prepare the data
def clean_data(df):
    # Handle missing values
    df = df.fillna({
        'children': 0,
        'country': 'Unknown'
    })
    
    # Convert date columns
    if 'booking_date' in df.columns:
        df['booking_date'] = pd.to_datetime(df['booking_date'])
    if 'arrival_date' in df.columns:
        df['arrival_date'] = pd.to_datetime(df['arrival_date'])
    
    # Create additional useful columns
    if 'booking_date' in df.columns and 'arrival_date' in df.columns:
        df['days_until_arrival'] = (df['arrival_date'] - df['booking_date']).dt.days
    
    if 'daily_rate' in df.columns and 'stays_in_nights' in df.columns:
        df['total_revenue'] = df['daily_rate'] * df['stays_in_nights']
    
    if 'adults' in df.columns and 'children' in df.columns:
        df['total_guests'] = df['adults'] + df['children']
        
    # Extract year and month for temporal analysis
    if 'booking_date' in df.columns:
        df['booking_year'] = df['booking_date'].dt.year
        df['booking_month'] = df['booking_date'].dt.month
        df['booking_quarter'] = df['booking_date'].dt.quarter
    
    return df

# Apply cleaning
df = clean_data(df)

# Display basic info about the dataset
print(f"Dataset shape: {df.shape}")
df.info()
df.head()

## Create User Statistics Section

In [ ]:
# Calculate key statistics for the dashboard

# Total bookings
total_bookings = len(df)

# Active bookings (not canceled)
if 'booking_status' in df.columns:
    active_bookings = df[df['booking_status'] != 'Canceled'].shape[0]
    cancellation_rate = df[df['booking_status'] == 'Canceled'].shape[0] / total_bookings * 100
else:
    active_bookings = "N/A"
    cancellation_rate = "N/A"

# Unique customers
if 'customer_id' in df.columns:
    unique_customers = df['customer_id'].nunique()
else:
    unique_customers = "N/A"

# Average stay duration
avg_stay = df['stays_in_nights'].mean()

# Average daily rate
avg_rate = df['daily_rate'].mean() if 'daily_rate' in df.columns else "N/A"

# Total revenue
total_revenue = df['total_revenue'].sum() if 'total_revenue' in df.columns else "N/A"

# Display statistics
print(f"Total Bookings: {total_bookings}")
print(f"Active Bookings: {active_bookings}")
print(f"Cancellation Rate: {cancellation_rate:.2f}%" if isinstance(cancellation_rate, float) else f"Cancellation Rate: {cancellation_rate}")
print(f"Unique Customers: {unique_customers}")
print(f"Average Stay Duration: {avg_stay:.2f} nights")
print(f"Average Daily Rate: ${avg_rate:.2f}" if isinstance(avg_rate, float) else f"Average Daily Rate: {avg_rate}")
print(f"Total Revenue: ${total_revenue:,.2f}" if isinstance(total_revenue, float) else f"Total Revenue: {total_revenue}")

In [ ]:
# Calculate monthly growth rate (if time data available)
if 'booking_date' in df.columns:
    # Group by month and count bookings
    monthly_bookings = df.groupby(pd.Grouper(key='booking_date', freq='M')).size().reset_index()
    monthly_bookings.columns = ['Month', 'Bookings']
    
    # Calculate month-over-month growth rate
    monthly_bookings['Growth_Rate'] = monthly_bookings['Bookings'].pct_change() * 100
    
    print("Monthly Booking Growth Rate:")
    print(monthly_bookings[['Month', 'Bookings', 'Growth_Rate']].tail(12))
else:
    print("Time-based data not available for growth rate calculation")

## Visualize Key Metrics

In [ ]:
# Create a figure for key metrics visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Booking Status Distribution", "Room Type Distribution", 
                   "Bookings by Hotel Type", "Top 10 Countries"),
    specs=[[{"type": "pie"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# 1. Booking Status Distribution (Pie Chart)
if 'booking_status' in df.columns:
    status_counts = df['booking_status'].value_counts()
    fig.add_trace(
        go.Pie(
            labels=status_counts.index, 
            values=status_counts.values,
            textinfo='percent+label',
            hole=0.3
        ),
        row=1, col=1
    )

# 2. Room Type Distribution (Bar Chart)
if 'room_type' in df.columns:
    room_counts = df['room_type'].value_counts()
    fig.add_trace(
        go.Bar(
            x=room_counts.index,
            y=room_counts.values,
            marker_color='darkblue'
        ),
        row=1, col=2
    )

# 3. Bookings by Hotel Type (Bar Chart)
if 'hotel_type' in df.columns:
    hotel_counts = df['hotel_type'].value_counts()
    fig.add_trace(
        go.Bar(
            x=hotel_counts.index,
            y=hotel_counts.values,
            marker_color='darkgreen'
        ),
        row=2, col=1
    )

# 4. Top 10 Countries (Bar Chart)
if 'country' in df.columns:
    country_counts = df['country'].value_counts().head(10)
    fig.add_trace(
        go.Bar(
            x=country_counts.values,
            y=country_counts.index,
            orientation='h',
            marker_color='darkred'
        ),
        row=2, col=2
    )

# Update layout
fig.update_layout(
    height=800, 
    width=1000,
    title_text="Key Booking Metrics",
    showlegend=False
)

fig.show()

In [ ]:
# Create time-based visualizations

# Check if we have time-based data
if 'booking_date' in df.columns:
    # Create a figure for time-based metrics
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=("Monthly Booking Trends", "Average Daily Rate Over Time"),
        specs=[[{"type": "scatter"}], [{"type": "scatter"}]],
        shared_xaxes=True,
        vertical_spacing=0.1
    )
    
    # 1. Monthly booking trends
    monthly_counts = df.groupby(pd.Grouper(key='booking_date', freq='M')).size()
    monthly_counts.index = monthly_counts.index.strftime('%Y-%m')
    
    fig.add_trace(
        go.Scatter(
            x=monthly_counts.index,
            y=monthly_counts.values,
            mode='lines+markers',
            name='Bookings',
            line=dict(color='royalblue', width=3)
        ),
        row=1, col=1
    )
    
    # 2. Average Daily Rate over time
    if 'daily_rate' in df.columns:
        monthly_rates = df.groupby(pd.Grouper(key='booking_date', freq='M'))['daily_rate'].mean()
        monthly_rates.index = monthly_rates.index.strftime('%Y-%m')
        
        fig.add_trace(
            go.Scatter(
                x=monthly_rates.index,
                y=monthly_rates.values,
                mode='lines+markers',
                name='Avg Rate',
                line=dict(color='firebrick', width=3)
            ),
            row=2, col=1
        )
    
    # Update layout
    fig.update_layout(
        height=600,
        width=1000,
        title_text="Booking Trends Over Time",
        xaxis2_tickangle=45
    )
    
    fig.show()
else:
    print("Time-based data not available for trend visualization")

In [ ]:
# Create a heatmap for booking patterns by day of week and month
if 'booking_date' in df.columns:
    # Extract day of week and month
    df['day_of_week'] = df['booking_date'].dt.day_name()
    df['month_name'] = df['booking_date'].dt.month_name()
    
    # Order days and months correctly
    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    months_order = ['January', 'February', 'March', 'April', 'May', 'June', 
                   'July', 'August', 'September', 'October', 'November', 'December']
    
    # Create a crosstab for the heatmap
    heatmap_data = pd.crosstab(
        index=pd.Categorical(df['day_of_week'], categories=days_order),
        columns=pd.Categorical(df['month_name'], categories=months_order)
    )
    
    plt.figure(figsize=(14, 8))
    sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='YlGnBu', linewidths=1, cbar_kws={'label': 'Bookings'})
    plt.title('Booking Patterns by Day of Week and Month', fontsize=16)
    plt.tight_layout()
    plt.show()

## Build Interactive Filters

In [ ]:
# Create interactive filters using ipywidgets

# Create widgets for filtering
if 'hotel_type' in df.columns:
    hotel_options = ['All'] + sorted(df['hotel_type'].unique().tolist())
    hotel_dropdown = widgets.Dropdown(
        options=hotel_options,
        value='All',
        description='Hotel Type:',
        style={'description_width': 'initial'},
        layout={'width': '250px'}
    )

if 'room_type' in df.columns:
    room_options = ['All'] + sorted(df['room_type'].unique().tolist())
    room_dropdown = widgets.Dropdown(
        options=room_options,
        value='All',
        description='Room Type:',
        style={'description_width': 'initial'},
        layout={'width': '250px'}
    )

# Create a date range selector if date column exists
if 'booking_date' in df.columns:
    min_date = df['booking_date'].min().date()
    max_date = df['booking_date'].max().date()
    
    date_range = widgets.DateRangeSlider(
        value=[min_date, max_date],
        min=min_date,
        max=max_date,
        step=1,
        description='Date Range:',
        style={'description_width': 'initial'},
        layout={'width': '500px'}
    )

# Display the widgets
filter_widgets = widgets.HBox([hotel_dropdown, room_dropdown]) if 'hotel_type' in df.columns and 'room_type' in df.columns else None

if filter_widgets:
    display(filter_widgets)
    
if 'booking_date' in df.columns:
    display(date_range)

In [ ]:
# Define a function to filter data based on selected values
def filter_data(hotel_type, room_type, date_range=None):
    filtered_df = df.copy()
    
    # Filter by hotel type
    if hotel_type != 'All' and 'hotel_type' in filtered_df.columns:
        filtered_df = filtered_df[filtered_df['hotel_type'] == hotel_type]
    
    # Filter by room type
    if room_type != 'All' and 'room_type' in filtered_df.columns:
        filtered_df = filtered_df[filtered_df['room_type'] == room_type]
    
    # Filter by date range
    if date_range is not None and 'booking_date' in filtered_df.columns:
        start_date = pd.Timestamp(date_range[0])
        end_date = pd.Timestamp(date_range[1])
        filtered_df = filtered_df[(filtered_df['booking_date'] >= start_date) & 
                                 (filtered_df['booking_date'] <= end_date)]
    
    return filtered_df

# Create a function to update visualizations based on filters
def update_visualization(hotel_type, room_type, date_range=None):
    filtered_data = filter_data(hotel_type, room_type, date_range)
    
    # Calculate metrics
    total_filtered = len(filtered_data)
    revenue = filtered_data['total_revenue'].sum() if 'total_revenue' in filtered_data.columns else 0
    avg_rate = filtered_data['daily_rate'].mean() if 'daily_rate' in filtered_data.columns else 0
    
    # Create visualization
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=("Booking Status", "Room Type Distribution"),
        specs=[[{"type": "pie"}, {"type": "bar"}]]
    )
    
    # Add booking status pie chart
    if 'booking_status' in filtered_data.columns:
        status_counts = filtered_data['booking_status'].value_counts()
        fig.add_trace(
            go.Pie(
                labels=status_counts.index, 
                values=status_counts.values,
                textinfo='percent+label'
            ),
            row=1, col=1
        )
    
    # Add room type bar chart
    if 'room_type' in filtered_data.columns:
        room_counts = filtered_data['room_type'].value_counts()
        fig.add_trace(
            go.Bar(
                x=room_counts.index,
                y=room_counts.values
            ),
            row=1, col=2
        )
    
    fig.update_layout(
        height=500,
        width=900,
        title_text=f"Filtered Results: {total_filtered} bookings, ${revenue:,.2f} revenue, ${avg_rate:.2f} avg. rate"
    )
    
    fig.show()

# Create interactive output for the filters
if 'hotel_type' in df.columns and 'room_type' in df.columns:
    @widgets.interact(
        hotel_type=hotel_dropdown,
        room_type=room_dropdown,
        date_range=date_range if 'booking_date' in df.columns else None
    )
    def update_dashboard(hotel_type, room_type, date_range=None):
        update_visualization(hotel_type, room_type, date_range)

## Generate Dashboard Layout

Using the components created above, we can now combine them into a cohesive dashboard layout. 

For a production environment, consider using Dash or Voila to render this notebook as an interactive dashboard.

In [ ]:
# Final dashboard layout demonstration
# Note: This is a simplified representation. For a production dashboard,
# consider using Dash, Panel, or Voila to render the dashboard.

from IPython.display import HTML, display

dashboard_html = """
<div style="padding: 20px; background-color: #f5f5f5; border-radius: 10px;">
    <h2 style="text-align: center; color: #1a5276;">Hotel Bookings Admin Dashboard</h2>
    <p style="text-align: center;">This dashboard provides an overview of hotel booking statistics and trends.</p>
    
    <div style="margin-top: 20px;">
        <h3>Dashboard Instructions:</h3>
        <ol>
            <li>Use the filters above to narrow down the data by hotel type, room type, and date range.</li>
            <li>Explore the visualizations that update dynamically based on your filter selections.</li>
            <li>For a more comprehensive dashboard experience, consider exporting this notebook to Dash or Voila.</li>
        </ol>
    </div>
    
    <div style="margin-top: 20px; text-align: center;">
        <p><strong>Key Metrics Summary:</strong></p>
        <p>Total Bookings: {total_bookings} | Average Stay: {avg_stay:.2f} nights | Total Revenue: ${total_revenue:,.2f}</p>
    </div>
</div>
""".format(
    total_bookings=total_bookings,
    avg_stay=avg_stay,
    total_revenue=total_revenue if isinstance(total_revenue, float) else 0
)

display(HTML(dashboard_html))

# Final note
print("""
To create a standalone dashboard application:
1. Install Dash: pip install dash
2. Convert these visualizations to Dash components
3. Define a layout and callbacks for interactivity
4. Run the Dash server to host your dashboard

Alternatively, you can use:
- Voila: voila admin_dashboard.ipynb
- Panel: pip install panel, then use pn.serve() to serve the dashboard
- Streamlit: Rewrite key components using Streamlit's API
""")

# Admin Dashboard for Hotel Booking System

This notebook creates an interactive admin dashboard to visualize and analyze key metrics for the hotel booking system.

## Import Required Libraries

We'll import the necessary libraries for data manipulation and visualization.

In [ ]:
# Import libraries for data manipulation
import pandas as pd
import numpy as np

# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import interactive dashboard libraries
import ipywidgets as widgets
from IPython.display import display

# Set default style for matplotlib
plt.style.use('seaborn-whitegrid')
sns.set_palette('viridis')

## Load and Prepare Data

In this section, we'll load the hotel booking dataset and prepare it for analysis by:
- Handling missing values
- Renaming columns for clarity
- Converting data types as needed

In [ ]:
# Load the hotel booking dataset
try:
    # Adjust the file path as needed
    df = pd.read_csv('../data/hotel_bookings.csv')
    print(f"Data loaded successfully with {df.shape[0]} rows and {df.shape[1]} columns")
except FileNotFoundError:
    print("File not found. Please check the file path.")
    
# Display the first few rows of the dataset
df.head()

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values[missing_values > 0])

# Basic data cleaning
df_clean = df.copy()

# Fill missing values appropriately
# For numerical columns, we can use the median
numeric_cols = df_clean.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    if df_clean[col].isnull().sum() > 0:
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# For categorical columns, we can use the mode
categorical_cols = df_clean.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df_clean[col].isnull().sum() > 0:
        df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])

# Verify missing values have been handled
print("\nRemaining missing values:")
print(df_clean.isnull().sum().sum())

In [ ]:
# Ensure proper data types
# Convert date columns to datetime format
if 'reservation_status_date' in df_clean.columns:
    df_clean['reservation_status_date'] = pd.to_datetime(df_clean['reservation_status_date'])

# Create additional features for analysis
if 'reservation_status_date' in df_clean.columns:
    df_clean['month'] = df_clean['reservation_status_date'].dt.month
    df_clean['year'] = df_clean['reservation_status_date'].dt.year

# Display updated dataset info
df_clean.info()

## Create User Statistics Section

In this section, we'll generate summary statistics about users, including:
- Total users/bookings
- Active bookings
- Booking trends over time

In [ ]:
# Calculate basic booking statistics
total_bookings = len(df_clean)
canceled_bookings = df_clean[df_clean['is_canceled'] == 1].shape[0]
active_bookings = total_bookings - canceled_bookings
cancelation_rate = (canceled_bookings / total_bookings) * 100

print(f"Total Bookings: {total_bookings}")
print(f"Active Bookings: {active_bookings}")
print(f"Canceled Bookings: {canceled_bookings}")
print(f"Cancelation Rate: {cancelation_rate:.2f}%")

# Create a summary DataFrame for quick access
summary_stats = pd.DataFrame({
    'Metric': ['Total Bookings', 'Active Bookings', 'Canceled Bookings', 'Cancelation Rate (%)'],
    'Value': [total_bookings, active_bookings, canceled_bookings, round(cancelation_rate, 2)]
})

summary_stats

In [ ]:
# Analyze user/booking growth trends over time
# Group by month and year to see booking patterns
if 'reservation_status_date' in df_clean.columns:
    # Create a time series of bookings
    booking_trends = df_clean.groupby([df_clean['reservation_status_date'].dt.year, 
                                      df_clean['reservation_status_date'].dt.month]).size().reset_index()
    booking_trends.columns = ['Year', 'Month', 'Bookings']
    
    # Create a proper datetime index for better plotting
    booking_trends['Date'] = pd.to_datetime(booking_trends['Year'].astype(str) + '-' + 
                                          booking_trends['Month'].astype(str) + '-01')
    
    print("Booking trends over time:")
    booking_trends.head()
else:
    # If reservation_status_date doesn't exist, try another approach
    print("Unable to analyze time trends due to missing date column.")

## Visualize Key Metrics

Now we'll create various visualizations to display important metrics such as:
- Revenue trends
- Booking distribution
- User engagement metrics

In [ ]:
# Create a function to generate a plotly dashboard card
def create_card(title, value, comparison=None, color="blue"):
    """
    Create a simple indicator card for the dashboard
    
    Parameters:
    title (str): The title of the card
    value (str/int/float): The main value to display
    comparison (str, optional): A comparison text (e.g., "+15% from last month")
    color (str): Color theme for the card
    
    Returns:
    fig: A plotly figure object
    """
    fig = go.Figure()
    
    fig.add_trace(go.Indicator(
        mode="number",
        value=value,
        title={"text": title},
        delta={"reference": 0, "valueformat": ".0f"},
        domain={"x": [0, 1], "y": [0, 1]}
    ))
    
    if comparison:
        fig.add_annotation(
            text=comparison,
            x=0.5,
            y=0.4,
            showarrow=False,
            font=dict(size=14)
        )
    
    fig.update_layout(
        height=200,
        margin=dict(l=10, r=10, t=30, b=10),
    )
    
    return fig

In [ ]:
# Create key metric cards
total_card = create_card("Total Bookings", total_bookings)
active_card = create_card("Active Bookings", active_bookings)
canceled_card = create_card("Canceled Bookings", canceled_bookings)

# Display the cards
total_card.show()
active_card.show()
canceled_card.show()

# Create visualizations for key metrics

# 1. Booking status distribution (pie chart)
labels = ['Active', 'Canceled']
values = [active_bookings, canceled_bookings]

fig1 = px.pie(
    names=labels, 
    values=values, 
    title="Booking Status Distribution",
    color_discrete_sequence=px.colors.qualitative.Set2
)
fig1.update_traces(textposition='inside', textinfo='percent+label')
fig1.update_layout(
    title_x=0.5,
    legend_title="Status",
    height=400
)
fig1.show()

In [ ]:
# 2. Booking trends over time (line chart)
if 'reservation_status_date' in df_clean.columns:
    fig2 = px.line(
        booking_trends, 
        x='Date', 
        y='Bookings',
        title='Booking Trends Over Time',
        labels={'Bookings': 'Number of Bookings', 'Date': 'Month/Year'},
    )
    fig2.update_layout(
        title_x=0.5,
        xaxis_title='Date',
        yaxis_title='Number of Bookings',
        height=400
    )
    fig2.show()

# 3. Hotel type distribution (bar chart)
if 'hotel' in df_clean.columns:
    hotel_counts = df_clean['hotel'].value_counts().reset_index()
    hotel_counts.columns = ['Hotel Type', 'Count']
    
    fig3 = px.bar(
        hotel_counts, 
        x='Hotel Type', 
        y='Count',
        title='Bookings by Hotel Type',
        color='Hotel Type',
        color_discrete_sequence=px.colors.qualitative.Safe
    )
    fig3.update_layout(
        title_x=0.5,
        xaxis_title='Hotel Type',
        yaxis_title='Number of Bookings',
        height=400
    )
    fig3.show()

In [ ]:
# 4. ADR (Average Daily Rate) Analysis
if 'adr' in df_clean.columns:
    # Calculate mean ADR by hotel type
    adr_by_hotel = df_clean.groupby('hotel')['adr'].mean().reset_index()
    adr_by_hotel.columns = ['Hotel Type', 'Average Daily Rate']
    
    fig4 = px.bar(
        adr_by_hotel,
        x='Hotel Type',
        y='Average Daily Rate',
        title='Average Daily Rate by Hotel Type',
        color='Hotel Type',
        color_discrete_sequence=px.colors.qualitative.Pastel
    )
    fig4.update_layout(
        title_x=0.5,
        xaxis_title='Hotel Type',
        yaxis_title='Average Daily Rate ($)',
        height=400
    )
    fig4.show()
    
    # ADR distribution
    fig5 = px.histogram(
        df_clean,
        x='adr',
        title='Distribution of Average Daily Rate',
        nbins=50,
        color_discrete_sequence=['indianred']
    )
    fig5.update_layout(
        title_x=0.5,
        xaxis_title='Average Daily Rate ($)',
        yaxis_title='Count',
        height=400
    )
    fig5.show()

## Build Interactive Filters

In this section, we'll create interactive filters to allow dynamic exploration of the data.
We'll use ipywidgets to create interactive elements like:
- Date range selectors
- Hotel type filters
- Customer segment filters

In [ ]:
# Create filter widgets
if 'hotel' in df_clean.columns:
    hotel_options = ['All'] + list(df_clean['hotel'].unique())
    hotel_dropdown = widgets.Dropdown(
        options=hotel_options,
        value='All',
        description='Hotel Type:',
        style={'description_width': 'initial'}
    )
    
# Date range filter if we have dates
date_range = widgets.DateRangeSlider(
    value=[pd.Timestamp('2015-01-01'), pd.Timestamp('2017-12-31')],
    min=pd.Timestamp('2015-01-01'),
    max=pd.Timestamp('2017-12-31'),
    step=1,
    description='Date Range:',
    style={'description_width': 'initial'},
    layout={'width': '500px'}
)

# Filter for market segment if it exists
if 'market_segment' in df_clean.columns:
    market_options = ['All'] + list(df_clean['market_segment'].unique())
    market_dropdown = widgets.Dropdown(
        options=market_options,
        value='All',
        description='Market Segment:',
        style={'description_width': 'initial'}
    )

# Create and display control panel
control_panel = widgets.VBox([
    widgets.HTML(value="<h3>Dashboard Filters</h3>"),
    hotel_dropdown,
    date_range,
    market_dropdown if 'market_segment' in df_clean.columns else widgets.HTML(value="")
])

display(control_panel)

In [ ]:
# Define filter application function
def apply_filters(hotel_type, date_range_start, date_range_end, market_segment=None):
    """
    Apply filters to the dataset based on selected values
    
    Parameters:
    hotel_type (str): Selected hotel type
    date_range (tuple): Start and end dates
    market_segment (str): Selected market segment
    
    Returns:
    pandas.DataFrame: Filtered dataset
    """
    filtered_df = df_clean.copy()
    
    # Apply hotel filter
    if hotel_type != 'All':
        filtered_df = filtered_df[filtered_df['hotel'] == hotel_type]
    
    # Apply date filter if date column exists
    if 'reservation_status_date' in filtered_df.columns:
        filtered_df = filtered_df[
            (filtered_df['reservation_status_date'] >= pd.to_datetime(date_range_start)) & 
            (filtered_df['reservation_status_date'] <= pd.to_datetime(date_range_end))
        ]
    
    # Apply market segment filter
    if market_segment is not None and market_segment != 'All':
        filtered_df = filtered_df[filtered_df['market_segment'] == market_segment]
    
    return filtered_df

# Create interactive function to update visualizations
def update_dashboard(hotel_type, date_range, market_segment=None):
    # Get start and end dates from range
    start_date, end_date = date_range
    
    # Apply filters
    filtered_data = apply_filters(hotel_type, start_date, end_date, market_segment)
    
    # Calculate metrics based on filtered data
    active = filtered_data[filtered_data['is_canceled'] == 0].shape[0]
    canceled = filtered_data[filtered_data['is_canceled'] == 1].shape[0]
    total = active + canceled
    
    # Create updated visualizations
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=["Booking Status Distribution", "Average Daily Rate", 
                      "Lead Time Distribution", "Bookings by Month"],
        specs=[[{"type": "pie"}, {"type": "bar"}],
               [{"type": "histogram"}, {"type": "bar"}]]
    )
    
    # Booking status pie chart
    fig.add_trace(
        go.Pie(labels=['Active', 'Canceled'], values=[active, canceled]),
        row=1, col=1
    )
    
    # ADR by hotel type
    if 'hotel' in filtered_data.columns and 'adr' in filtered_data.columns:
        adr_data = filtered_data.groupby('hotel')['adr'].mean().reset_index()
        fig.add_trace(
            go.Bar(x=adr_data['hotel'], y=adr_data['adr'], name='ADR'),
            row=1, col=2
        )
    
    # Lead time histogram
    if 'lead_time' in filtered_data.columns:
        fig.add_trace(
            go.Histogram(x=filtered_data['lead_time'], nbinsx=20),
            row=2, col=1
        )
    
    # Bookings by month
    if 'arrival_date_month' in filtered_data.columns:
        month_data = filtered_data['arrival_date_month'].value_counts().reset_index()
        month_data.columns = ['Month', 'Count']
        fig.add_trace(
            go.Bar(x=month_data['Month'], y=month_data['Count']),
            row=2, col=2
        )
    
    # Update layout
    fig.update_layout(height=800, title_text=f"Dashboard for {hotel_type} Hotels")
    
    return fig

In [ ]:
# Create an interactive output
output = widgets.Output()

# Define update function
def on_change(change):
    with output:
        # Clear previous output
        output.clear_output()
        
        # Get current filter values
        hotel_type = hotel_dropdown.value
        start_date, end_date = date_range.value
        market_segment = market_dropdown.value if 'market_segment' in df_clean.columns else None
        
        # Update dashboard with new filters
        updated_fig = update_dashboard(hotel_type, (start_date, end_date), market_segment)
        updated_fig.show()

# Register callbacks
hotel_dropdown.observe(on_change, names='value')
date_range.observe(on_change, names='value')
if 'market_segment' in df_clean.columns:
    market_dropdown.observe(on_change, names='value')

# Display initial dashboard
with output:
    initial_fig = update_dashboard('All', date_range.value)
    initial_fig.show()

# Display the output
display(output)

## Generate Dashboard Layout

In this section, we'll combine all the components created above into a cohesive dashboard layout.

In [ ]:
from IPython.display import HTML, display
import plotly.io as pio

def generate_final_dashboard():
    """
    Combine all dashboard elements into a cohesive layout
    """
    # Create header
    header = HTML("""
    <div style="background-color:#4472C4; color:white; padding:15px; text-align:center; margin-bottom:20px">
        <h1>Hotel Booking Admin Dashboard</h1>
        <p>An interactive dashboard for hotel booking management and analysis</p>
    </div>
    """)
    
    # Display the complete dashboard
    display(header)
    display(control_panel)
    display(output)
    
    # Display additional information and insights
    insights = HTML("""
    <div style="background-color:#F2F2F2; padding:15px; margin-top:20px; border-left:5px solid #4472C4">
        <h3>Key Insights</h3>
        <ul>
            <li>Use the filters above to explore different segments of the data</li>
            <li>The dashboard updates automatically when you change any filter</li>
            <li>You can export visualizations using the Plotly export button in the upper right corner of each chart</li>
            <li>For more detailed analysis, you can modify the code cells in this notebook</li>
        </ul>
    </div>
    """)
    
    display(insights)

# Generate the final dashboard
generate_final_dashboard()

## Conclusion and Next Steps

This dashboard provides a comprehensive overview of hotel booking data with interactive filtering capabilities. 
Here are some potential next steps to enhance this dashboard:

1. **Implement predictive analytics** - Add models to predict booking cancellations or occupancy rates
2. **Improve data refresh** - Set up automatic data updates through scheduled scripts
3. **Add user authentication** - Implement access controls for different admin roles
4. **Create downloadable reports** - Add functionality to export key findings as PDF or Excel reports
5. **Optimize performance** - If the dataset is large, implement data sampling or caching strategies

In [ ]:
# Save any modified data for later use
# df_clean.to_csv('../data/cleaned_hotel_bookings.csv', index=False)

print("Dashboard notebook completed!")

# Admin Dashboard

This notebook creates an interactive admin dashboard with user statistics and key metrics visualization.

## Import Required Libraries

In [ ]:
# Import libraries for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML
import datetime
import warnings

# Set visualization styles and ignore warnings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## Load and Prepare Data

In [ ]:
# Load the dataset
# Replace with your actual data file path
try:
    df = pd.read_csv('../data/hotel_bookings.csv')
    print(f"Data loaded successfully with {df.shape[0]} rows and {df.shape[1]} columns")
except FileNotFoundError:
    print("Data file not found. Please update the file path.")
    # Create sample data for demonstration
    df = pd.DataFrame({
        'booking_date': pd.date_range(start='2022-01-01', periods=1000),
        'user_id': np.random.randint(1, 500, 1000),
        'is_active': np.random.choice([True, False], 1000, p=[0.8, 0.2]),
        'region': np.random.choice(['North', 'South', 'East', 'West'], 1000),
        'revenue': np.random.uniform(100, 5000, 1000),
        'booking_status': np.random.choice(['Confirmed', 'Cancelled', 'No-show'], 1000, p=[0.7, 0.2, 0.1]),
        'customer_type': np.random.choice(['Transient', 'Contract', 'Group'], 1000),
        'room_type': np.random.choice(['Standard', 'Deluxe', 'Suite'], 1000),
        'length_of_stay': np.random.randint(1, 15, 1000)
    })
    print("Created sample data for demonstration")

In [ ]:
# Data cleaning and preparation
def clean_data(df):
    # Check for missing values
    print("Missing values before cleaning:")
    print(df.isnull().sum())
    
    # Handle missing values based on column type
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col].fillna('Unknown', inplace=True)
        else:
            df[col].fillna(df[col].median(), inplace=True)
    
    # Convert date columns to datetime if they exist
    date_cols = [col for col in df.columns if 'date' in col.lower()]
    for col in date_cols:
        try:
            df[col] = pd.to_datetime(df[col])
        except:
            print(f"Could not convert {col} to datetime")
    
    # Create month and year columns if date column exists
    if date_cols:
        main_date_col = date_cols[0]
        df['month'] = df[main_date_col].dt.month
        df['year'] = df[main_date_col].dt.year
    
    print("\nMissing values after cleaning:")
    print(df.isnull().sum())
    
    return df

df = clean_data(df)

# Display dataset information
print("\nDataset overview:")
df.info()

# Display first few rows of the cleaned dataset
df.head()

## Create User Statistics Section

In [ ]:
# Generate user statistics
def calculate_user_stats(dataframe):
    stats = {}
    
    # Total users
    stats['total_users'] = dataframe['user_id'].nunique()
    
    # Active users (if 'is_active' column exists)
    if 'is_active' in dataframe.columns:
        stats['active_users'] = dataframe[dataframe['is_active'] == True]['user_id'].nunique()
        stats['active_percentage'] = (stats['active_users'] / stats['total_users']) * 100
    
    # Bookings per user
    stats['avg_bookings_per_user'] = dataframe.shape[0] / stats['total_users']
    
    # User distribution by region (if region column exists)
    if 'region' in dataframe.columns:
        stats['users_by_region'] = dataframe.groupby('region')['user_id'].nunique().to_dict()
    
    # User growth over time (if date column exists)
    if 'booking_date' in dataframe.columns and isinstance(dataframe['booking_date'].iloc[0], pd.Timestamp):
        user_growth = dataframe.groupby(dataframe['booking_date'].dt.strftime('%Y-%m'))['user_id'].nunique()
        stats['user_growth'] = user_growth.to_dict()
        
    return stats

user_stats = calculate_user_stats(df)

# Display user statistics
print("User Statistics Summary")
print("-" * 30)
for key, value in user_stats.items():
    if not isinstance(value, dict):
        print(f"{key.replace('_', ' ').title()}: {value:.2f}" if isinstance(value, float) else f"{key.replace('_', ' ').title()}: {value}")

# Create HTML table for user statistics
if 'users_by_region' in user_stats:
    region_df = pd.DataFrame(list(user_stats['users_by_region'].items()), columns=['Region', 'Users'])
    display(HTML(region_df.to_html(index=False)))

## Visualize Key Metrics

In [ ]:
# Create a function to visualize user growth over time
def plot_user_growth():
    if 'user_growth' in user_stats:
        user_growth_df = pd.DataFrame(list(user_stats['user_growth'].items()), columns=['Period', 'Users'])
        user_growth_df = user_growth_df.sort_values('Period')
        
        plt.figure(figsize=(12, 6))
        plt.plot(user_growth_df['Period'], user_growth_df['Users'], marker='o', linestyle='-', color='#1f77b4')
        plt.title('User Growth Over Time', fontsize=16)
        plt.xlabel('Month', fontsize=12)
        plt.ylabel('Number of Unique Users', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    else:
        print("User growth data not available")

# Create a function to visualize booking status distribution
def plot_booking_status():
    if 'booking_status' in df.columns:
        plt.figure(figsize=(10, 6))
        status_counts = df['booking_status'].value_counts()
        colors = sns.color_palette('viridis', len(status_counts))
        
        plt.pie(status_counts, labels=status_counts.index, autopct='%1.1f%%', 
                startangle=90, colors=colors, wedgeprops=dict(width=0.5))
        plt.title('Booking Status Distribution', fontsize=16)
        plt.axis('equal')
        plt.tight_layout()
        plt.show()
    else:
        print("Booking status data not available")

# Create a function to visualize revenue metrics
def plot_revenue_metrics():
    if 'revenue' in df.columns:
        # Monthly revenue trend
        if 'booking_date' in df.columns and isinstance(df['booking_date'].iloc[0], pd.Timestamp):
            monthly_revenue = df.groupby(df['booking_date'].dt.strftime('%Y-%m'))['revenue'].sum().reset_index()
            monthly_revenue = monthly_revenue.sort_values('booking_date')
            
            plt.figure(figsize=(12, 6))
            plt.bar(monthly_revenue['booking_date'], monthly_revenue['revenue'], color='skyblue')
            plt.title('Monthly Revenue', fontsize=16)
            plt.xlabel('Month', fontsize=12)
            plt.ylabel('Revenue', fontsize=12)
            plt.xticks(rotation=45)
            plt.grid(axis='y', alpha=0.3)
            plt.tight_layout()
            plt.show()
        
        # Revenue by room type if available
        if 'room_type' in df.columns:
            plt.figure(figsize=(10, 6))
            room_revenue = df.groupby('room_type')['revenue'].sum().sort_values(ascending=False)
            
            sns.barplot(x=room_revenue.index, y=room_revenue.values, palette='viridis')
            plt.title('Revenue by Room Type', fontsize=16)
            plt.xlabel('Room Type', fontsize=12)
            plt.ylabel('Total Revenue', fontsize=12)
            plt.grid(axis='y', alpha=0.3)
            plt.tight_layout()
            plt.show()
    else:
        print("Revenue data not available")

# Visualize the metrics
plot_user_growth()
plot_booking_status()
plot_revenue_metrics()

## Build Interactive Filters

In [ ]:
# Interactive dashboard with Plotly and ipywidgets
def build_interactive_dashboard():
    # Check required columns
    required_cols = ['booking_date', 'revenue', 'region', 'booking_status']
    missing_cols = [col for col in required_cols if col not in df.columns]
    
    if missing_cols:
        print(f"Cannot create dashboard. Missing columns: {missing_cols}")
        return
    
    if not isinstance(df['booking_date'].iloc[0], pd.Timestamp):
        print("Cannot create dashboard. 'booking_date' is not a datetime column.")
        return
    
    # Prepare data
    df['month_year'] = df['booking_date'].dt.strftime('%Y-%m')
    
    # Get unique values for filters
    regions = sorted(df['region'].unique())
    statuses = sorted(df['booking_status'].unique())
    
    # Create widgets
    region_dropdown = widgets.SelectMultiple(
        options=regions,
        value=[regions[0]],
        description='Region:',
        disabled=False
    )
    
    status_dropdown = widgets.SelectMultiple(
        options=statuses,
        value=statuses,
        description='Status:',
        disabled=False
    )
    
    date_range_slider = widgets.SelectionRangeSlider(
        options=sorted(df['month_year'].unique()),
        index=(0, len(df['month_year'].unique())-1),
        description='Date Range:',
        disabled=False
    )
    
    # Create update function
    def update_dashboard(region, status, date_range):
        # Filter data based on selections
        filtered_df = df[
            (df['region'].isin(region)) &
            (df['booking_status'].isin(status)) &
            (df['month_year'] >= date_range[0]) &
            (df['month_year'] <= date_range[1])
        ]
        
        if filtered_df.empty:
            print("No data matches the selected filters.")
            return
            
        # Create subplot figure
        fig = make_subplots(rows=2, cols=2,
                           subplot_titles=('Revenue by Region', 'Booking Status Distribution', 
                                          'Monthly Revenue Trend', 'Customer Type Distribution'),
                           specs=[[{'type': 'bar'}, {'type': 'pie'}],
                                 [{'type': 'scatter'}, {'type': 'pie'}]])
        
        # Plot 1: Revenue by Region
        region_revenue = filtered_df.groupby('region')['revenue'].sum().reset_index()
        fig.add_trace(go.Bar(x=region_revenue['region'], 
                            y=region_revenue['revenue'], 
                            marker_color='skyblue',
                            name='Revenue'),
                     row=1, col=1)
        
        # Plot 2: Booking Status Distribution
        status_counts = filtered_df['booking_status'].value_counts()
        fig.add_trace(go.Pie(labels=status_counts.index, 
                            values=status_counts.values, 
                            hole=0.3,
                            name='Booking Status'),
                     row=1, col=2)
        
        # Plot 3: Monthly Revenue Trend
        monthly_revenue = filtered_df.groupby('month_year')['revenue'].sum().reset_index()
        fig.add_trace(go.Scatter(x=monthly_revenue['month_year'], 
                                y=monthly_revenue['revenue'],
                                mode='lines+markers',
                                marker_color='green',
                                name='Monthly Revenue'),
                     row=2, col=1)
        
        # Plot 4: Customer Type Distribution if available
        if 'customer_type' in filtered_df.columns:
            customer_counts = filtered_df['customer_type'].value_counts()
            fig.add_trace(go.Pie(labels=customer_counts.index, 
                                values=customer_counts.values, 
                                hole=0.3,
                                name='Customer Type'),
                         row=2, col=2)
        
        # Update layout
        fig.update_layout(height=800, title_text="Interactive Hotel Booking Dashboard",
                         showlegend=False)
        
        # Display the dashboard
        fig.show()
        
        # Display key metrics
        total_revenue = filtered_df['revenue'].sum()
        total_bookings = filtered_df.shape[0]
        avg_revenue_per_booking = total_revenue / total_bookings if total_bookings > 0 else 0
        
        metrics_html = f"""
        <div style="background-color:#f6f6f6; padding:10px; border-radius:5px; margin-top:20px;">
            <h3>Key Metrics Summary</h3>
            <p><b>Total Revenue:</b> ${total_revenue:,.2f}</p>
            <p><b>Total Bookings:</b> {total_bookings:,}</p>
            <p><b>Average Revenue per Booking:</b> ${avg_revenue_per_booking:,.2f}</p>
        </div>
        """
        
        display(HTML(metrics_html))
    
    # Create interactive output
    out = widgets.interactive_output(update_dashboard, 
                                    {'region': region_dropdown, 
                                     'status': status_dropdown,
                                     'date_range': date_range_slider})
    
    # Display controls and output
    controls = widgets.VBox([region_dropdown, status_dropdown, date_range_slider])
    display(widgets.HBox([controls, out]))

# Run the interactive dashboard
build_interactive_dashboard()

## Generate Summary Reports

In [ ]:
# Generate summary reports
def generate_summary_report():
    # Create a summary dataframe
    summary = {}
    
    # User metrics
    summary['Total Users'] = [user_stats['total_users']]
    if 'active_users' in user_stats:
        summary['Active Users'] = [user_stats['active_users']]
        summary['Active User Percentage'] = [f"{user_stats['active_percentage']:.2f}%"]
    
    summary['Average Bookings per User'] = [f"{user_stats['avg_bookings_per_user']:.2f}"]
    
    # Booking metrics
    summary['Total Bookings'] = [df.shape[0]]
    
    # Revenue metrics if available
    if 'revenue' in df.columns:
        summary['Total Revenue'] = [f"${df['revenue'].sum():,.2f}"]
        summary['Average Revenue per Booking'] = [f"${df['revenue'].mean():,.2f}"]
    
    # Status metrics if available
    if 'booking_status' in df.columns:
        for status in df['booking_status'].unique():
            status_count = df[df['booking_status'] == status].shape[0]
            status_pct = (status_count / df.shape[0]) * 100
            summary[f'{status} Bookings'] = [f"{status_count} ({status_pct:.2f}%)"]
    
    # Create the summary dataframe
    summary_df = pd.DataFrame(summary)
    
    # Display the summary
    print("HOTEL BOOKING DASHBOARD - EXECUTIVE SUMMARY")
    print("=" * 50)
    display(summary_df.T.rename(columns={0: 'Value'}))
    
    # Export options
    print("\nExport options:")
    
    # Export to CSV
    try:
        csv_path = 'hotel_booking_summary_report.csv'
        summary_df.to_csv(csv_path)
        print(f"✓ Summary exported to CSV: {csv_path}")
    except:
        print("✗ Failed to export to CSV")
    
    # Create a detailed summary by month if date column exists
    if 'booking_date' in df.columns and isinstance(df['booking_date'].iloc[0], pd.Timestamp):
        try:
            # Monthly metrics
            monthly_df = df.groupby(df['booking_date'].dt.strftime('%Y-%m')).agg({
                'user_id': 'nunique',
                'booking_date': 'count'
            }).rename(columns={
                'user_id': 'Unique Users',
                'booking_date': 'Total Bookings'
            })
            
            if 'revenue' in df.columns:
                monthly_revenue = df.groupby(df['booking_date'].dt.strftime('%Y-%m'))['revenue'].agg(['sum', 'mean'])
                monthly_revenue.columns = ['Total Revenue', 'Average Revenue']
                monthly_df = pd.concat([monthly_df, monthly_revenue], axis=1)
            
            print("\nMonthly Performance Summary:")
            display(monthly_df)
            
            # Export monthly summary to CSV
            monthly_csv_path = 'hotel_booking_monthly_summary.csv'
            monthly_df.to_csv(monthly_csv_path)
            print(f"✓ Monthly summary exported to CSV: {monthly_csv_path}")
        except:
            print("✗ Failed to generate monthly summary")

# Generate the summary report
generate_summary_report()

## Conclusion

This admin dashboard provides a comprehensive view of hotel booking data with:

1. **User Statistics** - Total users, active users, and user growth trends
2. **Key Metrics Visualization** - Revenue trends, booking status distribution, and other important KPIs
3. **Interactive Filters** - Dynamic filtering by region, date range, and booking status
4. **Summary Reports** - Executive summary and detailed monthly performance reports

The dashboard can be enhanced by:
- Adding more advanced analytics like forecasting and predictive models
- Implementing user segmentation and customer lifetime value analysis
- Creating automated alerts for key metric changes
- Adding geographic visualization if location data is available